In [1]:
import os
os.chdir('../')

import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os

!nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"]="6"

Wed May 24 18:01:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    Off  | 00000000:1B:00.0 Off |                    0 |
| 30%   27C    P8    13W / 230W |  14334MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A5000    Off  | 00000000:1C:00.0 Off |                  Off |
| 30%   

### Hyperparams

In [3]:
n_mels = 8
n_outputs = 61
n_frames = 400
sr = 24000
fps = 30
train_csv_files = ['/Storage/speech/face/rvh_viseme2_1_iPhone_raw_30fps.csv',
                   '/Storage/speech/face/rvh_viseme2_2_iPhone_raw_30fps.csv',
                   '/Storage/speech/face/rvh_viseme2_3_iPhone_raw_30fps.csv',
                   '/Storage/speech/face/rvh_viseme2_4_iPhone_raw_30fps.csv',
                  ]
train_wav_files = ['/Storage/speech/face/rvh_viseme2_1_iPhone.wav',
                   '/Storage/speech/face/rvh_viseme2_2_iPhone.wav',
                   '/Storage/speech/face/rvh_viseme2_3_iPhone.wav',
                   '/Storage/speech/face/rvh_viseme2_4_iPhone.wav',
                  ]

test_csv_files = ['/Storage/speech/face/MySlate_6_박수철의_iPhone_30fps.csv']
test_wav_files = ['/Storage/speech/face/MySlate_6_박수철의_iPhone.wav']

### Model

In [4]:
from model.model_vqvae import Model
from utils.util import *
from tensorboardX import SummaryWriter

step = 0
device = 'cuda:0'

# Model
model = Model(in_dim=n_mels, out_dim=n_outputs, K=16, latent_dim=128)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
print('done')

done


### Load

In [5]:
save_dir = '/data/scpark/save/lips/train05.24-2/'
!mkdir -p $save_dir
!ls -lt $save_dir

writer = SummaryWriter(save_dir)

if False:
    step, model, _, optimizer = load(save_dir, 1000, model, None, optimizer)

total 0


In [6]:
import torch
from data.dataset import LipsDataset, Collate

dataset = LipsDataset(train_wav_files, train_csv_files, n_frames, n_mels=n_mels, sr=sr, fps=fps)
print('train dataset :', len(dataset))
train_loader = torch.utils.data.DataLoader(dataset, num_workers=1, shuffle=True, batch_size=8, 
                                           collate_fn=Collate(n_frames, n_mels))
print(train_loader)

dataset = LipsDataset(test_wav_files, test_csv_files, n_frames, n_mels=n_mels, sr=sr, fps=fps)
print('test dataset :', len(dataset))
test_loader = torch.utils.data.DataLoader(dataset, num_workers=1, shuffle=True, batch_size=8,
                                           collate_fn=Collate(n_frames, n_mels))
print(test_loader)

train dataset : 28308
test dataset : 1413


In [ ]:
from IPython import display
import librosa.display
import matplotlib.pyplot as plt

isnan = False
while True:
    if isnan:
        break
    for batch in train_loader:
        inputs = batch['inputs'].transpose(1, 2).to(device)
        targets = batch['outputs'].transpose(1, 2).to(device)
        
        model.train()
        model.zero_grad()
        outputs = model(inputs, targets)
        
        print(step)
        loss = 0
        for key in outputs.keys():
            if 'loss' in key:
                loss += outputs[key]
                print(key, outputs[key].item())
        if torch.isnan(loss):
            isnan = True
            break
        loss.backward()
        optimizer.step()
        print(step, loss.item())
        
        if step % 100 == 0:
            writer.add_scalar('train_loss', loss.item(), step)
        
        if step % 100 == 0:
            display.clear_output()
            
            losses = []
            for i, batch in enumerate(test_loader):
                if i >= 10:
                    break
                    
                inputs = batch['inputs'].transpose(1, 2).to(device)
                targets = batch['outputs'].transpose(1, 2).to(device)

                model.eval()
                with torch.no_grad():
                    outputs = model(inputs, targets)
                    
                loss = 0
                for key in outputs.keys():
                    if 'loss' in key:
                        loss += outputs[key]
                print('test :', i, loss.item())
                losses.append(loss)        
            
            test_loss = torch.stack(losses).mean().item()
            print('test_loss :', test_loss)
            writer.add_scalar('test_loss', test_loss, step)

        if step % 1000 == 0:
            save(save_dir, step, model, None, optimizer)
    
        step += 1

test : 0 2.7978241443634033
test : 1 2.797598361968994
test : 2 2.7966384887695312
test : 3 2.7996463775634766
test : 4 2.795571804046631
test : 5 2.798635244369507
test : 6 2.7987468242645264
test : 7 2.795344829559326
test : 8 2.79875111579895
test : 9 2.7983510494232178
test_loss : 2.79771089553833
saved /data/scpark/save/lips/train05.24-2/save_0
1
auto_encoding_loss 0.08627477288246155
commit_loss 0.0005327654653228819
zi_prediction_loss 2.7832040786743164
1 2.870011568069458
2
auto_encoding_loss 0.07567403465509415
commit_loss 0.0004689121269620955
zi_prediction_loss 2.7821156978607178
2 2.8582587242126465
3
auto_encoding_loss 0.07216548919677734
commit_loss 0.0004201845731586218
zi_prediction_loss 2.7756175994873047
3 2.848203182220459
4
auto_encoding_loss 0.06116654723882675
commit_loss 0.0004177737282589078
zi_prediction_loss 2.7651853561401367
4 2.8267695903778076
5
auto_encoding_loss 0.06085701286792755
commit_loss 0.0004173984343651682
zi_prediction_loss 2.760072946548462
5 

59 2.485335350036621
60
auto_encoding_loss 0.021328747272491455
commit_loss 0.004323875531554222
zi_prediction_loss 2.4034316539764404
60 2.429084300994873
61
auto_encoding_loss 0.020977837964892387
commit_loss 0.004366101697087288
zi_prediction_loss 2.399923801422119
61 2.4252676963806152
62
auto_encoding_loss 0.02090943418443203
commit_loss 0.0042777047492563725
zi_prediction_loss 2.3887364864349365
62 2.413923740386963
63
auto_encoding_loss 0.019749293103814125
commit_loss 0.004276012536138296
zi_prediction_loss 2.3918638229370117
63 2.415889024734497
64
auto_encoding_loss 0.020656095817685127
commit_loss 0.004246552940458059
zi_prediction_loss 2.3805289268493652
64 2.4054315090179443
65
auto_encoding_loss 0.019153611734509468
commit_loss 0.004273752216249704
zi_prediction_loss 2.293269157409668
65 2.3166964054107666
66
auto_encoding_loss 0.021408459171652794
commit_loss 0.004231734666973352
zi_prediction_loss 2.3680670261383057
66 2.393707275390625
67
auto_encoding_loss 0.019239328